In [1]:
#importing the necessary dependencies
import os
from langchain.llms import GooglePalm


In [2]:
#loading g the llm
llm=GooglePalm(google_api_key=os.environ['GOOGLE_API_KEY'],temperature=0.6)

c:\Users\ADMIN\Documents\Data Analytics\Machine Learning\NLP\NLP Projects\Text_to_SQL\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Testing the llm
poem = llm('write a poem for my love for babushka')
print(poem)

Babushka, my dear babushka,
I love you so much.
Your warm smile and gentle touch
Bring me such joy.

You are always there for me,
No matter what.
You listen to my worries
And help me through my troubles.

You are the wisest person I know,
And I cherish your advice.
You have taught me so much about life,
And I am a better person because of you.

I am so grateful to have you in my life,
Babushka.
You are my best friend,
And I love you more than words can say.

Thank you for always being there for me,
Babushka.
I love you.


In [4]:
#connecting to SQL database
from langchain.utilities import SQLDatabase

#db_password: Do Not Use Special Characters
db_user = "root"
db_password = "root1234"
db_host = "localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [5]:
#using SQL chain
from langchain_experimental.sql import SQLDatabaseChain

#Put verbose=true to view the generated SQL statements
db_chain = SQLDatabaseChain.from_llm(llm,db,verbose=True)
#Question
qns1 = db_chain.run("How many t-shirts do we have left for nike in extra small size and white color?")



> Entering new SQLDatabaseChain chain...


How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(14,)]
Answer:14
> Finished chain.


In [6]:
qns1

'14'

In [7]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('396'),)]
Answer:396
> Finished chain.


In [8]:
qns2 = db_chain.run("How much is the value of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the value of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('22190'),)]
Answer:22190
> Finished chain.


The LLM will be able to figure out that value of inventory = SUM(price * stock_quantity)

In [9]:
#Question 3
#qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")

From above we can observe a case where the model hallucinates where the model uses columns that DO NOT exist in the database. We can use few shot learning to improve on this.

In [10]:
#Running the SQL code directly
qns3 = db_chain.run("""SELECT SUM(t.price*(100-coalesce(d.pct_discount,0))* t.stock_quantity) AS revenue
FROM t_shirts AS t
LEFT JOIN discounts AS d ON t.t_shirt_id=d.t_shirt_id
WHERE brand='Levi';""")



> Entering new SQLDatabaseChain chain...
SELECT SUM(t.price*(100-coalesce(d.pct_discount,0))* t.stock_quantity) AS revenue
FROM t_shirts AS t
LEFT JOIN discounts AS d ON t.t_shirt_id=d.t_shirt_id
WHERE brand='Levi';
SQLQuery:SELECT SUM(t.price*(100-coalesce(d.pct_discount,0))* t.stock_quantity) AS revenue
FROM t_shirts AS t
LEFT JOIN discounts AS d ON t.t_shirt_id=d.t_shirt_id
WHERE brand='Levi';
SQLResult: [(Decimal('2071800.00'),)]
Answer:2071800.00
> Finished chain.


In [11]:
#If I sell all Levis tshirts how much revenue will I generate
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('20718'),)]
Answer:20718
> Finished chain.


In [12]:
# Number of White levi's t shirts
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand='Levi' AND color='White'
SQLResult: [(41,), (14,)]
Answer:41
> Finished chain.


In [13]:
# Number of White levi's t shirts
qns5 = db_chain.run("How much white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How much white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE color = 'White' AND brand = 'Levi'
SQLResult: [(41,), (14,)]
Answer:41
> Finished chain.


From above the llm model failed to figure out that the query is asking for the sum of all the quantities of Levis thata are white

In [14]:
#Using the correct query
qns5 = db_chain.run("""SELECT SUM(stock_quantity) 
FROM t_shirts
WHERE brand='Levi' AND color='White';""")



> Entering new SQLDatabaseChain chain...
SELECT SUM(stock_quantity) 
FROM t_shirts
WHERE brand='Levi' AND color='White';
SQLQuery:SELECT SUM(stock_quantity) 
FROM t_shirts
WHERE brand='Levi' AND color='White';
SQLResult: [(Decimal('55'),)]
Answer:55
> Finished chain.


## Few Shot Learning
We will utlize few shot learning to fix the issues we have seen so far

In [15]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

In [16]:
qns1,qns2,qns3,qns4,qns5

('14', '22190', '2071800.00', '20718', '55')

### Creating Semantic Similarity Based example selector
- create embedding on the few_shots: Using Google Gen Embeddings
- Store the embeddings in a vector store: FAISS index
- Retrieve the the top most Semantically close example from the vector store

In [17]:
#Using Google 
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings=GoogleGenerativeAIEmbeddings(model='models/embedding-001')

In [18]:
#Need to combine all values in the few shot examples into one sentence
# creating a blob of all the sentences
to_vectorize = [" ".join(example.values()) for example in few_shots]

In [19]:
#Using the FAISS vector database
from langchain.vectorstores import FAISS #vector embeddngs

#generating a vector store: 
vector_store=FAISS.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [20]:
type(vector_store)

langchain.vectorstores.faiss.FAISS

In [21]:
#Checking sematic similarity: # Helping to pull similar looking queries
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore = vector_store,
    k=2, #number of examples
)

example_selector.select_examples({"Question": "How many Adidas T shirts are left in my store?"})

[{'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query',
  'Answer': '14'},
 {'Question': "How many white color Levi's shirt I have?",
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of the SQL query',
  'Answer': '55'}]

### Setting up Prompts

Adding custom prompts to provide better context to the LLM

In [22]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt
from langchain.prompts.prompt import PromptTemplate
#adding mysql prompt
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [23]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [24]:
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

"""

In [25]:
#Example prompt format
example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [26]:
# Entire Prompt
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [27]:
#Chain_
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [28]:
new_chain("How many white color Levi's shirt I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:

SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('55'),)]
Answer:55
> Finished chain.


{'query': "How many white color Levi's shirt I have?", 'result': '55'}

In [29]:
new_chain("What is the number of white color Nike shirts do I have in stock?")



> Entering new SQLDatabaseChain chain...
What is the number of white color Nike shirts do I have in stock?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White'
SQLResult: [(14,), (64,), (54,), (37,)]
Answer:64
> Finished chain.


{'query': 'What is the number of white color Nike shirts do I have in stock?',
 'result': '64'}

In [30]:
new_chain("What is the total number of small size white color Nike shirts do I have in stock?")



> Entering new SQLDatabaseChain chain...
What is the total number of small size white color Nike shirts do I have in stock?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'S'
SQLResult: [(Decimal('64'),)]
Answer:64
> Finished chain.


{'query': 'What is the total number of small size white color Nike shirts do I have in stock?',
 'result': '64'}

In [32]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('26527.350000'),)]
Answer:26527.35
> Finished chain.


{'query': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '26527.35'}

In [ ]:
new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Van Huesen'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('31263.800000'),)]
Answer:31263.8
> Finished chain.


{'query': 'If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '31263.8'}

In [33]:
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:SELECT sum(price * stock_quantity) FROM t_shirts WHERE brand = 'Van Huesen'
SQLResult: [(Decimal('34082'),)]
Answer:34082
> Finished chain.


'34082'